In [8]:
with open("input.txt") as f:
    input_data = f.readlines()

In [17]:
line = "".join(input_data)
line

'Valve TM has flow rate=0; tunnels lead to valves KF, AA\nValve LG has flow rate=8; tunnels lead to valves DD, UA\nValve IZ has flow rate=20; tunnels lead to valves LY, XC\nValve XF has flow rate=0; tunnels lead to valves PB, QD\nValve FE has flow rate=0; tunnels lead to valves ZW, KF\nValve ZP has flow rate=0; tunnels lead to valves MT, AI\nValve CL has flow rate=0; tunnels lead to valves JN, AI\nValve UA has flow rate=0; tunnels lead to valves VW, LG\nValve VP has flow rate=0; tunnels lead to valves MB, GU\nValve KY has flow rate=0; tunnels lead to valves BZ, CJ\nValve AI has flow rate=11; tunnels lead to valves TL, GG, CL, ZP, MM\nValve GD has flow rate=0; tunnels lead to valves KB, QK\nValve GU has flow rate=14; tunnels lead to valves ZK, VP\nValve RO has flow rate=0; tunnels lead to valves KS, TJ\nValve VW has flow rate=0; tunnels lead to valves UA, KS\nValve YE has flow rate=24; tunnel leads to valve DP\nValve AA has flow rate=0; tunnels lead to valves TL, ZU, TM, RL, BZ\nValve R

In [22]:
import time,sys
import re
from collections import deque
import networkx as nx
start=time.time()

inf = 'input.txt'
G = nx.Graph()
valves = dict()
with open(inf) as fi:
    lines = fi.read()[:-1].strip(',').split('\n')
    for line in lines:
        l = line.strip(',')
        a = l.split(' ')[9:]
        for i  in range(len(a)):
            a[i] = a[i][:2]
        rate = [int(x) for x in re.findall(r'[-]?\d+',line)]
        #valves['AA'] = 0
        if rate[0] > 0:
            valves[l.split(' ')[1]]= rate[0]
        for x in a:
            G.add_edge(l.split(' ')[1],x)
            G.add_edge(x,l.split(' ')[1])
print(valves)
distances = dict()
for v1 in G.nodes():
    for v2 in G.nodes():
        if v1 != v2:
            distances[(v1,v2)] = len(nx.shortest_path(G,v1,v2))-1

def findnextValve(s,openvalves):
    nodes = deque([(s,set())])
    posibleValves = set()
    while nodes:
        start, visited = nodes.popleft()
        visited.add(start)
        for n in valves.keys():
            if n not in visited and n != start:
                nodes.append((n,visited))
                if n not in openvalves.keys() and n != s:
                    posibleValves.add((n,distances[(s,n)]))
    return posibleValves

def release(openvalves,maxmin):
    pressure = 0
    for valve in openvalves.keys():
        pressure += valves[valve]*(maxmin-openvalves[valve])
    return pressure

def releasepressure(start,openvalves,maxmin, E = False):
    valve = start
    paths = deque([(valve, 0, openvalves,False,0,0)])
    finalpaths = deque([])
    while paths:
        valve, dist, openvalves, moving, mins , released = paths.popleft()
        if moving:
            if mins +dist< maxmin+1:
                paths.append((valve, 0, openvalves, not moving , mins+dist,0))
            else:
                #if maxmin == 26 and not E:
                    #pathsE = releasepressure('AA',openvalves,26,True)
                    #for _,_,o,_,_,_ in pathsE:
                        #openvalves = openvalves | o
                        #finalpaths.append((valve, 0, openvalves, not moving, mins,release(openvalves,maxmin)))
                #else:
                    #finalpaths.append((valve, 0, openvalves, not moving, mins,release(openvalves,maxmin)))
                finalpaths.append((valve, 0, openvalves, not moving, mins,release(openvalves,maxmin)))
        else:
            mins +=1
            if valve != start:
                openvalves[valve]=mins-1
            nextvalves = findnextValve(valve,openvalves)
            if len(nextvalves)>0:
                for nvalve, dist in nextvalves:
                    paths.append((nvalve,dist, openvalves.copy(),not moving, mins,0))
            else:
                #if maxmin == 26 and not E:
                    #pathsE = releasepressure('AA',openvalves,26,True)
                    #for _,_,o,_,_,_ in pathsE:
                        #openvalves = openvalves | o
                        #finalpaths.append((valve, 0, openvalves, not moving, mins,release(openvalves,maxmin)))
                #else:
                    #finalpaths.append((valve, 0, openvalves, not moving, mins,release(openvalves,maxmin)))
                finalpaths.append((valve, 0, openvalves, not moving, maxmin,release(openvalves,maxmin)))
    return finalpaths

paths = releasepressure('AA',dict(),30)
rpressures = set()
for valve,_,openvalves,moving,mins,releasedpressure in paths:
    if releasedpressure > 1000:
        rpressures.add(releasedpressure)
print('part1:', max(rpressures))

print('part2 -finding paths...')
paths = releasepressure('AA',dict(),26)
rpressures = deque([])
for valve,_,openvalves,moving,mins,releasedpressure in paths:
    if releasedpressure > 1000:
        rpressures.append((openvalves,releasedpressure))
print('part2 - find maximum of elephant and player')

maxpressures= set()

i=0
for o1,r1 in rpressures:
    i+=1
    for o2, r2 in rpressures:
        if o1.keys().isdisjoint(o2.keys()):
            #print(r1+r2,o1,o2)
            maxpressures.add(r1+r2)
    if i %1000==0:
        print(i)
print('part2:', max(maxpressures))

end=time.time()

print(round(end-start,5))

{'LG': 8, 'IZ': 20, 'AI': 11, 'GU': 14, 'YE': 24, 'QK': 15, 'EK': 22, 'CJ': 10, 'MB': 18, 'KS': 13, 'CU': 19, 'PB': 5, 'KF': 7, 'TJ': 3, 'KB': 21}
part1: 1724
part2 -finding paths...
part2 - find maximum of elephant and player
1000
2000
3000
4000
5000
6000
7000
8000
part2: 2283
48.24444
